In [4]:
%load_ext autoreload

In [5]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../../..')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save, to_string

## Load Neural Data

In [4]:
session = 'Mary-Anesthesia-20160809-01'
# session = 'MrJones-Anesthesia-20160113-01.mat'
# filename = fr'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/{session}.mat'
filename = fr'/om/user/eisenaj/ChaoticConsciousness/data/propofolPuffTone/{session}.mat'
print("Loading data ...")
start = time.process_time()
electrode_info, lfp, lfp_schema, session_info, spike_times, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'unitInfo'], verbose=False)
spike_times = spike_times[0]
dt = lfp_schema['smpInterval'][0]
T = lfp.shape[0]

# f = h5py.File(filename, 'r')
# airPuff_binary, audio_binary = get_binary_stimuli(f)

print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")

Loading data ...
Data loaded (took 102.96 seconds)


# VAR

In [5]:
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)
data_path = f"/om/user/eisenaj/ChaoticConsciousness/results/propofol/VAR"
os.makedirs(data_path, exist_ok=True)

PCA_dim = -1
end_step = T - int(30/dt) # take off 30 seconds

# end_step = 15000
# for window in [2.5]:
# for window in [0.15, 0.25, 0.5, 1, 2.5, 5]:
# for window in [0.25, 0.5, 1, 2.5, 5]:
for window in [0.75, 1.5, 2, 3, 4]:
    stride = window
    run_data = {}
    
    # ================
    # RUN PARAMS
    # ================
    run_data['run_params'] = dict(
        window=window,
        stride=stride,
        PCA_dim=PCA_dim,
        end_step=end_step,
        unit_indices={}
    )
    
    run_data['results'] = {}
    
    # ================
    # AREA BY AREA
    # ================
    
#     for area in np.unique(electrode_info['area']):
#         areas = [area]
#         unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]

#         data = lfp[:end_step, unit_indices]
#         print(f"window = {window}, area = {area}")
#         results = compute_eigs(data, dt, window, stride, return_sigma_norms=True, return_A=True, PCA_dim=PCA_dim, verbose=True)
#         results['data_sigma_norms'] = get_data_sigma_norms(data, dt, window, stride)
#         del data
#         for key in results.keys():
#             if len(results[key].shape) > 1:
#                 results[key] = list(results[key])
#         results = pd.DataFrame(results)
        
#         run_data['results'][area] = results  
#         run_data['run_params']['unit_indices'][area] = unit_indices
    
    # ================
    # ALL AREAS
    # ================
    
    print(f"window = {window}, all areas")
    areas = np.unique(electrode_info['area'])
    unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]
    data = lfp[:end_step, unit_indices]
    results = compute_eigs(data, dt, window, stride, return_sigma_norms=True, return_A=True, PCA_dim=PCA_dim, verbose=True)
    results['data_sigma_nor/om/user/eisenms'] = get_data_sigma_norms(data, dt, window, stride)
    del data
    for key in results.keys():
        if len(results[key].shape) > 1:
            results[key] = list(results[key])
    results = pd.DataFrame(results)
    
    run_data['results']['all'] = results
    run_data['run_params']['unit_indices']['all'] = unit_indices

    save(run_data, os.path.join(data_path, f"VAR_{os.path.basename(filename)[:-4]}_window_{window}_stride_{stride}_{timestamp}"))

window = 0.75, all areas


  0%|          | 0/9407 [00:00<?, ?it/s]

window = 1.5, all areas


  0%|          | 0/4703 [00:00<?, ?it/s]

window = 2, all areas


  0%|          | 0/3527 [00:00<?, ?it/s]

window = 3, all areas


  0%|          | 0/2351 [00:00<?, ?it/s]

window = 4, all areas


  0%|          | 0/1763 [00:00<?, ?it/s]

In [51]:
var_data = "/om/user/eisenaj/ChaoticConsciousness/results/propofolWakeup/VAR/VAR_Mary-Anesthesia-20170203-02_window_2.5_stride_2.5/FEF/window_1163"

In [52]:
var_window = load(var_data)

In [53]:
var_window

{'start_ind': 2907500,
 'start_time': 2907.5,
 'end_ind': 2910000,
 'end_time': 2910.0,
 'explained_variance': None,
 'A_mat': array([[ 0.97285727,  0.03403564, -0.12228417, ..., -0.01783858,
          0.05799666, -0.04499522],
        [ 0.00343862,  1.00343882, -0.10138505, ..., -0.01211935,
          0.02196587,  0.0281815 ],
        [-0.03049425,  0.04135523,  0.73415301, ..., -0.02613038,
          0.03051656, -0.02521762],
        ...,
        [-0.01505086,  0.04804948, -0.65656648, ...,  0.96052345,
          0.11066385, -0.20543974],
        [-0.05086159,  0.0839901 , -0.62497407, ..., -0.01833342,
          1.03159911, -0.11728186],
        [-0.07075263,  0.08353061, -0.6327468 , ..., -0.05546332,
          0.03639933,  0.94705173]]),
 'A_mat_with_bias': array([[ 0.00149468,  0.00145243,  0.0019745 , ...,  0.00396099,
          0.00392612,  0.0044    ],
        [ 0.97285727,  0.00343862, -0.03049425, ..., -0.01505086,
         -0.05086159, -0.07075263],
        [ 0.03403564,  1

In [54]:
len(var_window['criticality_inds'])

54

In [55]:
54 + 63 + 63 + 64 + 20

264